In [45]:
import numpy as np
import numpy.linalg as nl

def turn_to_dict(file_name):
    beacon_dict = {}

    with open(file_name, 'r') as packs:
        for pack in packs:
            pack = pack[0: len(pack) - 1]
            space_loc = pack.find(' ')
            beacon = pack[0: space_loc]
            ri = float(pack[space_loc + 1:])

            if beacon not in beacon_dict:
                beacon_dict[beacon] = [ri]
            else:
                beacon_dict[beacon].append(ri)

    return beacon_dict

In [46]:
def get_alpha(beacon_dict):
    #ssvs => signal strength values
    
    num_beacon = len(beacon_dict)
    alphas = np.zeros(num_beacon)
    
    for beacon in beacon_dict:
        beacon_index = int(beacon[1:]) - 1
        ssvs = np.array(beacon_dict[beacon])
        num_ssv = len(ssvs)
        
        shift_vec = np.arange(num_ssv - 1) + 1
        ssvs_shift = ssvs[shift_vec]
        ssvs_trunc = ssvs[0: len(ssvs) - 1]
    
        sbar = np.mean(ssvs)
        r1 = abs(np.sum((ssvs_shift - sbar)*(ssvs_trunc - sbar))) / (np.sum((ssvs - sbar) ** 2))
        alphas[beacon_index] = r1
        
    return alphas
        

In [47]:
def train(state_files, num_beacon):
    # Currently fake data
    # alpha => [alpha_b1, alpha_b2, ..., alpha_bn]

    num_state = len(state_files)
    state_map = np.zeros((num_state, num_beacon, 2))
    
    for file, si in zip(state_files, range(num_state)):
        beacon_dict = turn_to_dict(file)
        alphas = get_alpha(beacon_dict)
      
        for rssis in beacon_dict:
            alpha = alphas[int(rssis[1:])-1]
            arr = beacon_dict[rssis]
            state_map[si][int(rssis[1:])-1][0] = np.mean(arr)
            state_map[si][int(rssis[1:])-1][1] = np.sqrt(((1 + alpha) / (1 - alpha))*np.var(arr))

    return state_map

In [48]:
def process_test(test_name, num_beacon):
    # Turn original file to numpy array with [avg_b1_rssi, avg_b2_rssi, ..., avg_bn_rssi]

    beacon_dict = turn_to_dict(test_name)
    avg = np.zeros(num_beacon)

    for bi in beacon_dict:
        index = int(bi[1:])
        avg[index - 1] = np.mean(beacon_dict[bi])

    return avg

In [49]:
def test(state_map, test_rssi, state_loc):
    # return state index with max liklihood
    # state_map => (state, beacon, (mu, sigma))
    # state_loc => (state, (x, y))
    # test_rssi => [avg_rssi]

    probs = []

    for i in range(state_map.shape[0]):
        map_mu = state_map[i][:, 0]
        map_sigma = state_map[i][:, 1]
        sum_log = - np.sum(np.square(test_rssi - map_mu) / (2 * map_sigma ** 2)) - \
                  np.sum(np.log(map_sigma))
        probs.append(1 / np.exp(-sum_log))
        
    probs = np.asarray(probs).reshape(state_map.shape[0], 1)
    probs = probs / np.sum(probs)
    res = np.sum(probs * state_loc, axis = 0)
    
    return res

In [50]:
def ss_compensator(prev_state, test_file, state_loc, state_map, thred = 2, d = 0.05):
    # thred => max distance per second
    # prev_state, pred_state => numpy (x, y) in meter metric
    # d => perturbation fraction
    
    pred_state = np.array([])
    min_dist = 0
    final_pert = -1
    test_rssi = process_test(test_file, 2)
    max_index = np.argmax(test_rssi)
    
    for pert in [-0.05, 0, 0.05]:
        pert_arr = np.zeros(len(test_rssi))
        pert_arr[max_index] = pert
       
        cur_pred = test(state_map, test_rssi + pert_arr, state_loc)
        cur_dist = nl.norm(prev_state - cur_pred)
        
        if final_pert == -1 or min_dist > cur_dist:
            final_pert = pert
            pred_state = cur_pred
            min_dist = cur_dist
    
    print("pert: " + str(final_pert))
     
    return pred_state

In [51]:
def main():
    state_loc = np.array([[0, 1], [0, 2]])
    state_files = ['s1', 's2']
    state_map = train(state_files, 2)
    res1 = ss_compensator(state_loc[0], 's1', state_loc, state_map)
    res2 = ss_compensator(state_loc[1], 's2', state_loc, state_map)
   
    print(res1, res2)

main()

[-0.01976617 -0.36615778]
[-0.06976617 -0.36615778]
[-0.01976617 -0.36615778]
[ 0.03023383 -0.36615778]
pert: 0
[1.22400118 0.94410251]
[1.17400118 0.94410251]
[1.22400118 0.94410251]
[1.27400118 0.94410251]
pert: 0.05
[0.         1.28918069] [0.         1.65724603]
